In [1]:
#imports
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
file = open("Metamorphosis.txt", "r", encoding = "utf8")

In [3]:
#pre-processing the data
lines = []

for i in file:
    lines.append(i)
    
len(lines)
data = ""

for i in lines:
    data = ' '. join(lines)
    
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')
len(data)

139064

In [4]:
#Toeknize the data
#nltk.download('punkt')
tokens = word_tokenize(data)
train_len=2
text_sequences=[]

for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

sequences={}
count = 1

for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]]=count
        count += 1

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)

#vocbulary size
vocabulary_size = len(tokenizer.word_counts) + 1
n_sequences = np.empty([len(sequences), train_len], dtype='int32')

for i in range(len(sequences)):
    n_sequences[i]=sequences[i]
    
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes = vocabulary_size)
seq_len = train_inputs.shape[1]

In [12]:
#model build
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from tensorflow import keras

model = keras.models.load_model('mymodel.h5')
# model = Sequential()
# model.add(Embedding(vocabulary_size, seq_len, input_length = seq_len))
# model.add(LSTM(50, return_sequences=True))
# model.add(LSTM(50))
# model.add(Dense(50, activation='relu'))
# model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())

#compiling the network
# model.compile(loss='categorical_crossentropy', optimizer ='adam', metrics=['accuracy'])
# model.fit(train_inputs, train_targets, epochs=500,verbose=1)
# model.save("mymodel.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 1)              3064      
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 50)             10400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 3064)              156264    
Total params: 192,478
Trainable params: 192,478
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#predicting words
from keras.preprocessing.sequence import pad_sequences

input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)

for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
    pred_word=tokenizer.index_word[i]
    print("Next word suggestion:", pred_word)